# Introduction to Cascaded Classification System for Intent Recognition

#### In the realm of natural language processing (NLP), effective intent recognition plays a pivotal role in enhancing user interaction with chatbots, virtual assistants, and automated customer support systems. This project implements a two-stage classification system that categorizes user queries into distinct intents based on their content.

#### The classification process is divided into two stages:

#### ****1. Symptom Checker vs. Non-Symptom Checker****: In the initial stage, a binary classifier, utilizing the BERT architecture, determines whether a given query pertains to a "Symptom Checker" or a "Non-Symptom Checker." This differentiation allows the system to streamline the subsequent classification process.

#### ****2. Intent Classification****: If a query is classified as "Non-Symptom Checker," a second classifier further categorizes it into specific intents, such as "Treatment Information," "Patient Support," "FAQs," and "Appointment Scheduling." This hierarchical approach enables the model to provide more accurate and contextually relevant responses to user inquiries.

#### The project leverages the BERT (Bidirectional Encoder Representations from Transformers) model for its powerful contextual understanding, ensuring high accuracy in intent classification. By utilizing this two-stage classification system, the project aims to improve user experience in healthcare-related applications by efficiently addressing queries based on their specific intent.

#### **********************************************************************************************************************************************

### Imports the Required Libraries

- **transformers**: For using BERT for tokenization and model building.
- **torch**: For tensor operations and model training.
- **pandas**: For data manipulation and reading CSV files.
- **sklearn**: For splitting datasets and evaluating models.
- **numpy**: For numerical operations.
- **joblib**: For saving model ojects.


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import joblib

### Load the data

loads the dataset containing queries and their corresponding intents into a Pandas DataFrame. The file path should be adjusted according to your directory structure.

In [5]:
df = pd.read_csv("C:/Users/augus/PycharmProjects/image_captioning/RAG DEMO/faq_queries_intents.csv")

### Label Encoding for the Intents

The following line encodes the intents into numeric labels using `pd.factorize()`, which assigns a unique integer to each unique intent. This creates a new column `label` for machine learning purposes:

In [7]:
df['label'], intent_labels = pd.factorize(df['intent'])

####  Define Symptom Checker and Non-Symptom Checker Categories

Here, two categories for classification are defined:

- **Symptom Checker** : Contains queries related to symptoms.
- **Non-Symptom Checker** : Contains other types of queries.
A new binary label is created that assigns 0 for symptom checker intents and 1 for non-symptom checker intents.

In [10]:
symptom_checker = ['Symptom_Checker']
non_symptom_checker = ['Treatment_Information', 'Patient_Support', 'FAQs', 'Appointment_Scheduling']

# Binary classification: Symptom Checker vs. Non-Symptom Checker
df['binary_label'] = df['intent'].apply(lambda x: 0 if x in symptom_checker else 1)

#### Split the Dataset 

 splits the dataset into training and testing sets for the binary classification model. The test size is set to 20%, and a random state is specified for reproducibility.

In [11]:
X_train_route, X_test_route, y_train_route, y_test_route = train_test_split(df['query'], df['binary_label'], test_size=0.2, random_state=42)

####  Load BERT Tokenizer and Model for Binary Classification

The BERT tokenizer and model are loaded from the Hugging Face Transformers library. The model is configured for binary classification with `num_labels=2`.

In [12]:
tokenizer_route = BertTokenizer.from_pretrained('bert-base-uncased')
model_route = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Check for GPU Availability

checks if a GPU is available for faster training and moves the model to the appropriate device (CPU or GPU). The tokenizer and model are then saved for future use.

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_route.to(device)

# Save tokenizer and model after training (optional)
tokenizer_route.save_pretrained("route_tokenizer")
model_route.save_pretrained("route_model")

#### Filter Out Non-Symptom Checker Data

filters the DataFrame to get only non-symptom checker data and applies label encoding. It then splits the non-symptom checker data into training and testing sets.

In [13]:
non_symptom_df = df[df['binary_label'] == 1]
non_symptom_df['label'], non_symptom_labels = pd.factorize(non_symptom_df['intent'])

# Split non-symptom checker data
X_train_non, X_test_non, y_train_non, y_test_non = train_test_split(non_symptom_df['query'], non_symptom_df['label'], test_size=0.2, random_state=42)

#### Load BERT Tokenizer and Model for Non-Symptom Checker Classification

Similar to the previous tokenizer and model setup, this block loads a new BERT tokenizer and model for the non-symptom checker classification task. The number of labels corresponds to the unique intents in the non-symptom checker category.

In [15]:
tokenizer_non_symptom = BertTokenizer.from_pretrained('bert-base-uncased')
model_non_symptom = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(non_symptom_labels))

tokenizer_non_symptom.save_pretrained("non_symptom_tokenizer")
model_non_symptom.save_pretrained("non_symptom_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Define the Training Function

The function `train_bert_model` trains a BERT model using the provided training data. The process includes:

1. **Initializing the Optimizer and Loss Function**:
   - The AdamW optimizer and the cross-entropy loss function are initialized to optimize the model's parameters during training.

2. **Iterating through the Dataset**:
   - The training process involves iterating through the dataset for a specified number of epochs, allowing the model to learn from the data over multiple passes.

3. **Dividing the Data into Batches**:
   - The data is divided into batches for training, which helps in managing memory and improving training efficiency.

4. **Tokenizing the Input Sentences**:
   - Input sentences are tokenized using the BERT tokenizer, converting text into a format suitable for the model.

5. **Performing Forward and Backward Passes**:
   - A forward pass is conducted to compute the model's output and the loss. Subsequently, a backward pass updates the model's weights based on the computed loss.

6. **Printing the Average Loss**:
   - The average loss is printed after each epoch, providing feedback on the model's learning progress and performance.


In [21]:
def train_bert_model(model, tokenizer, X_train, y_train, epochs=3, batch_size=16):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    model.train()

    # Convert y_train to a numpy array (this will fix the ValueError)
    y_train = np.array(y_train)

    for epoch in range(epochs):
        total_loss = 0
        for i in range(0, len(X_train), batch_size):
            batch_sentences = X_train[i:i + batch_size]
            batch_labels = torch.tensor(y_train[i:i + batch_size]).to(device)

            # Tokenize inputs
            inputs = tokenizer(batch_sentences.tolist(), return_tensors='pt', truncation=True, padding=True,
                               max_length=64).to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(**inputs).logits
            loss = loss_fn(outputs, batch_labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1} completed with average loss: {total_loss / len(X_train)}")

#### Train the Models

call the training function for both the binary and non-symptom checker models using their respective training datasets.

In [22]:
train_bert_model(model_route, tokenizer_route, X_train_route, y_train_route)

train_bert_model(model_non_symptom, tokenizer_non_symptom, X_train_non, y_train_non)

Epoch 1 completed with average loss: 0.00019462801171206388
Epoch 2 completed with average loss: 4.2668648287905795e-06
Epoch 3 completed with average loss: 2.3692240212388344e-06
Epoch 1 completed with average loss: 0.031784947188743105
Epoch 2 completed with average loss: 0.013067292687814869
Epoch 3 completed with average loss: 0.007467058579539413


#### Save the Trained Models and Tokenizers

After training, the models and tokenizers are saved for later use. This allows you to avoid retraining when making predictions in the future.

In [26]:
tokenizer_route.save_pretrained("route_tokenizer")
model_route.save_pretrained("route_model")

tokenizer_non_symptom.save_pretrained("non_symptom_tokenizer")
model_non_symptom.save_pretrained("non_symptom_model")

#### Reload the Models and Tokenizers for Inference

This block reloads the saved models and tokenizers for use in making predictions.

In [27]:
tokenizer_route = BertTokenizer.from_pretrained("route_tokenizer")
model_route = BertForSequenceClassification.from_pretrained("route_model").to(device)

tokenizer_non_symptom = BertTokenizer.from_pretrained("non_symptom_tokenizer")
model_non_symptom = BertForSequenceClassification.from_pretrained("non_symptom_model").to(device)

#### Define the Prediction Function

This function predicts the intent of a given sentence by first checking if it belongs to the symptom checker category. If it does, it returns "Symptom_Checker"; otherwise, it predicts the specific intent from the non-symptom checker category.

In [28]:
def predict_intent(sentence):
    model_route.eval()
    model_non_symptom.eval()

    # Tokenize and predict using the routing classifier (binary)
    inputs_route = tokenizer_route(sentence, return_tensors='pt', truncation=True, padding=True, max_length=64).to(device)
    with torch.no_grad():
        logits_route = model_route(**inputs_route).logits
    predicted_binary_label = torch.argmax(logits_route, dim=1).cpu().item()

    if predicted_binary_label == 0:
        return "Symptom_Checker"
    else:
        # Predict using the non-symptom classifier (multiclass)
        inputs_non_symptom = tokenizer_non_symptom(sentence, return_tensors='pt', truncation=True, padding=True, max_length=64).to(device)
        with torch.no_grad():
            logits_non_symptom = model_non_symptom(**inputs_non_symptom).logits
        predicted_label = torch.argmax(logits_non_symptom, dim=1).cpu().item()
        return non_symptom_labels[predicted_label]

#### Test the Model with Sample Queries

This block tests the model by passing sample queries through the prediction function and printing the predicted intents.

In [29]:
sample_queries = [
    "What are the symptoms of flu?",
    "How can I book an appointment?",
    "What treatment is available for diabetes?",
    "Can you help me with my insurance questions?"
]

for query in sample_queries:
    print(f"Query: '{query}' -> Predicted Intent: {predict_intent(query)}")

Query: 'What are the symptoms of flu?' -> Predicted Intent: Treatment_Information
Query: 'How can I book an appointment?' -> Predicted Intent: appointment_scheduling
Query: 'What treatment is available for diabetes?' -> Predicted Intent: Treatment_Information
Query: 'Can you help me with my insurance questions?' -> Predicted Intent: Patient_Support


#### Evaluating Predictions with Accuracy and F1-Score


After making predictions, we evaluate them using **Accuracy** and **F1-Score**:

- **Accuracy**:
  - Measures the percentage of correct predictions made by the model.

- **F1-Score**:
  - Combines precision and recall into a single metric, making it useful for assessing model performance on imbalanced datasets.

The `classification_report` function provides detailed metrics for both classes, including:
- **Precision**: The ratio of correctly predicted positive observations to the total predicted positives.
- **Recall**: The ratio of correctly predicted positive observations to the all observations in actual class.
- **F1-Score**: The weighted average of precision and recall, providing a balance between the two.

This comprehensive evaluation helps in understanding the model's strengths and weaknesses across different classes.


## Conclusion
In this notebook, we implemented a two-stage intent recognition system using BERT, capable of classifying user queries into symptom checker and non-symptom checker categories. The models were trained on a dataset of queries and intents, and their performance can be evaluated using further metrics as needed.